In [1]:
!pip install sparknlp

     |████████████████████████████████| 51kB 3.9MB/s 


In [5]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 66kB/s 
     |████████████████████████████████| 204kB 42.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=23d3ff82499bddb12736792dccb290fd725aa76a7383e7c74402580714638659
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [38]:
!pip install nltk

In [2]:
import pandas as pd
import numpy as np

In [19]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [21]:
config = SparkSession.builder.master("local[*]").config("spark.executor.memory", "70g").config("spark.driver.memory", "50g").config("spark.memory.offHeap.enabled",True).config("spark.memory.offHeap.size","16g").appName("clinical").getOrCreate()

In [22]:
data = config.read.option('Header',True).csv('/content/data.csv')

In [23]:
data.show()

+------+-------------+--------------------+--------------------+--------------------+
|    id|     category|        sub-category|               title|            abstract|
+------+-------------+--------------------+--------------------+--------------------+
|N55528|    lifestyle|     lifestyleroyals|the brands queen ...|shop the notebook...|
|N19639|       health|          weightloss|50 worst habits f...|these seemingly h...|
|N61837|         news|           newsworld|the cost of trump...|lt ivan molchanet...|
|N53526|       health|              voices|i was an nba wife...|i felt like i was...|
|N38324|       health|             medical|how to get rid of...|they seem harmles...|
| N2073|       sports|        football_nfl|should nfl be abl...|several fines cam...|
|N49186|      weather|   weathertopstories|it s been orlando...|there won t be a ...|
|N59295|         news|           newsworld|chile three die i...|three people have...|
|N24510|entertainment|              gaming|best ps5 ga

In [33]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import * 

In [34]:
spark = sparknlp.start()

In [40]:
import nltk

In [41]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
from nltk.corpus import stopwords
eng_stopwords = stopwords.words('english')

In [43]:
documentAssembler = DocumentAssembler().setInputCol('title').setOutputCol('document')

tokenizer = Tokenizer().setInputCols(['document']).setOutputCol('tokenized')

lemmatizer = LemmatizerModel.pretrained().setInputCols(['tokenized']).setOutputCol('lemmatized')

stopwords_cleaner = StopWordsCleaner().setInputCols(['lemmatized']).setOutputCol('no_stop_lemmatized').setStopWords(eng_stopwords)

finisher = Finisher().setInputCols(['no_stop_lemmatized'])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


In [44]:
pipes = Pipeline(stages=[documentAssembler,tokenizer,lemmatizer,stopwords_cleaner,finisher])

In [45]:
df = data
data_fit = pipes.fit(df)
data_fin = data_fit.transform(df)

In [47]:
data_fin.show()

+------+-------------+--------------------+--------------------+--------------------+---------------------------+
|    id|     category|        sub-category|               title|            abstract|finished_no_stop_lemmatized|
+------+-------------+--------------------+--------------------+--------------------+---------------------------+
|N55528|    lifestyle|     lifestyleroyals|the brands queen ...|shop the notebook...|       [brand, queen, el...|
|N19639|       health|          weightloss|50 worst habits f...|these seemingly h...|       [50, bad, habit, ...|
|N61837|         news|           newsworld|the cost of trump...|lt ivan molchanet...|       [cost, trump, aid...|
|N53526|       health|              voices|i was an nba wife...|i felt like i was...|       [nba, wife, affec...|
|N38324|       health|             medical|how to get rid of...|they seem harmles...|       [get, rid, skin, ...|
| N2073|       sports|        football_nfl|should nfl be abl...|several fines cam...|   

In [24]:
from pyspark.ml import Pipeline

In [54]:
df = data_fin.toPandas()

In [55]:
df.to_csv('data_spark.csv',index=None)